This project is also an opportunity to illustrate how to use pandas new API to register custom DataFrame accessors. This API allows you to build custom functions for working with pandas DataFrames and Series and could be really useful for building out your own library of custom pandas accessor functions.

# sidetable
At its core, sidetable is a super-charged version of pandas value_counts with a little bit of crosstab mixed in. For instance, let’s look at some data on School Improvement Grants so we can see how sidetable can help us explore a new data set and figure out approaches for more complex analysis.

The only external dependency is pandas version >= 1.0. Make sure it is installed, then install sidetable:

In [ ]:
python -m pip install sidetable

once side table is install, you need to import it to get the pandas accessor registered.

In [1]:
import pandas as pd
import sidetable as st

In [2]:
df = pd.read_csv(r"pythonsidetables.csv", encoding = 'unicode_escape', index_col= 0)
df.head(5)


,School Name,City,State,District Name,Model Selected,Award_Amount,Region
ï»¿,,,,,,,
0,HOGARTH KINGEEKUK MEMORIAL SCHOOL,SAVOONGA,AK,BERING STRAIT SCHOOL DISTRICT,Transformation,471014,West
1,AKIACHAK SCHOOL,AKIACHAK,AK,YUPIIT SCHOOL DISTRICT,Transformation,520579,West
2,GAMBELL SCHOOL,GAMBELL,AK,BERING STRAIT SCHOOL DISTRICT,Transformation,449592,West
3,BURCHELL HIGH SCHOOL,WASILLA,AK,MATANUSKA-SUSITNA BOROUGH SCHOOL DISTRICT,Transformation,641184,West
4,AKIAK SCHOOL,AKIAK,AK,YUPIIT SCHOOL DISTRICT,Transformation,399686,West


In [3]:
df.stb.freq(['State']).head(5)

,State,count,percent,cumulative_count,cumulative_percent
0,CA,92,12.153236,92,12.153236
1,FL,71,9.379128,163,21.532365
2,PA,58,7.661823,221,29.194188
3,OH,35,4.623514,256,33.817701
4,MO,32,4.227213,288,38.044914


This example shows that CA occurs 91 times and represents 12.02% of the total number of schools. If you include FL in the counts, you now have 162 total schools that represent 21.4% of the total.

In [4]:
df.stb.freq(['State'], thresh=50)

,State,count,percent,cumulative_count,cumulative_percent
0,CA,92,12.153236,92,12.153236
1,FL,71,9.379128,163,21.532365
2,PA,58,7.661823,221,29.194188
3,OH,35,4.623514,256,33.817701
4,MO,32,4.227213,288,38.044914
5,MI,28,3.698811,316,41.743725
6,GA,26,3.434610,342,45.178336
7,NY,25,3.302510,367,48.480845
8,others,390,51.519155,757,100.000000


This is handy. Now we can see that 8 states contributed almost 50% of the total and all the other states account for the remainder.

If we want, we can rename the catch-all category using other_label

In [5]:
df.stb.freq(['State'], thresh=50, other_label='Rest of states')

,State,count,percent,cumulative_count,cumulative_percent
0,CA,92,12.153236,92,12.153236
1,FL,71,9.379128,163,21.532365
2,PA,58,7.661823,221,29.194188
3,OH,35,4.623514,256,33.817701
4,MO,32,4.227213,288,38.044914
5,MI,28,3.698811,316,41.743725
6,GA,26,3.434610,342,45.178336
7,NY,25,3.302510,367,48.480845
8,Rest of states,390,51.519155,757,100.000000


One of the useful features of sidetable is that it can group columns together to further understand the distribution. For instance, what if we want to see how the various “Transformation Models” are applied across Regions?

In [6]:
df.stb.freq(['Region', 'Model Selected'])

,Region,Model Selected,count,percent,cumulative_count,cumulative_percent
0,South,Transformation,185,24.765730,185,24.765730
1,West,Transformation,142,19.009371,327,43.775100
2,Midwest,Transformation,111,14.859438,438,58.634538
3,Northeast,Transformation,102,13.654618,540,72.289157
4,West,Turnaround,49,6.559572,589,78.848728
5,South,Turnaround,44,5.890228,633,84.738956
6,Midwest,Turnaround,43,5.756359,676,90.495315
7,Northeast,Turnaround,25,3.346720,701,93.842035
8,South,Restart,11,1.472557,712,95.314592
9,Northeast,Restart,9,1.204819,721,96.519411


This view is a quick way to understand the interaction and distribution of the various data elements. I find that this is an easy way to explore data and get some insights that might warrant further analysis. A table like this is also easy to share with others since it is relatively simple to understand.

In [7]:
df['Award_Amount'] = pd.to_numeric(df['Award_Amount'])

In [8]:
df.stb.freq(['Region'], value ='Award_Amount')

,Region,Award_Amount,percent,cumulative_Award_Amount,cumulative_percent
0,South,117467481,37.314735,117467481,37.314735
1,West,74418552,23.639807,191886033,60.954542
2,Midwest,65736175,20.881762,257622208,81.836304
3,Northeast,57179654,18.163696,314801862,100.000000


This view gives us insight that the Northeast has the least amount of dollars spent on these projects and that 37% of the total spend went to schools in the South region.

Finally, we can look at the types of models selected and determine the 80/20 breakdown of the allocated dollars:

In [9]:
df.stb.freq(['Region', 'Model Selected'],
             value='Award_Amount', thresh=82,
             other_label='Remaining')

,Region,Model Selected,Award_Amount,percent,cumulative_Award_Amount,cumulative_percent
0,South,Transformation,88680032,28.170110,88680032,28.170110
1,West,Transformation,56207890,17.855006,144887922,46.025116
2,Midwest,Transformation,48702505,15.470844,193590427,61.495960
3,Northeast,Transformation,41263161,13.107661,234853588,74.603621
4,South,Turnaround,22531412,7.157331,257385000,81.760952
5,Remaining,Remaining,57416862,18.239048,314801862,100.000000


Comparing above results with pandas cross table.

In [10]:
pd.crosstab(df['Region'],df['Model Selected'], values =df['Award_Amount'], aggfunc='sum')

Model Selected,Closure,Restart,Transformation,Turnaround
Region,,,,
Midwest,86872,1397735,48702505,15549063
Northeast,508773,5728010,41263161,9679710
South,354323,5901714,88680032,22531412
West,272520,2245146,56207890,15692996


One of sidetable’s goals is that its output is easy to interpret. If you would like to leverage pandas style functions to format your output for improved readability, sidetable can format Percentage and Amount columns to be more readable. This is not used by default but can be seen by passing style=True to the function

In [11]:
df.stb.freq(['Region', 'Model Selected'], value='Award_Amount',thresh=82, style=True)

,Region,Model Selected,Award_Amount,percent,cumulative_Award_Amount,cumulative_percent
0,South,Transformation,"88,680,032",28.17%,"88,680,032",28.17%
1,West,Transformation,"56,207,890",17.86%,"144,887,922",46.03%
2,Midwest,Transformation,"48,702,505",15.47%,"193,590,427",61.50%
3,Northeast,Transformation,"41,263,161",13.11%,"234,853,588",74.60%
4,South,Turnaround,"22,531,412",7.16%,"257,385,000",81.76%
5,others,others,"57,416,862",18.24%,"314,801,862",100.00%


In [12]:
df.stb.missing()

,missing,total,percent
Region,10,757,1.321004
School Name,0,757,0.000000
City,0,757,0.000000
State,0,757,0.000000
District Name,0,757,0.000000
Model Selected,0,757,0.000000
Award_Amount,0,757,0.000000


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757 entries, 0 to 830
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   School Name     757 non-null    object
 1   City            757 non-null    object
 2   State           757 non-null    object
 3   District Name   757 non-null    object
 4   Model Selected  757 non-null    object
 5   Award_Amount    757 non-null    int64 
 6   Region          747 non-null    object
dtypes: int64(1), object(6)
memory usage: 47.3+ KB


if you look at both the above function missing and info are providing same output but missing function on the sidetable is easier to read and understand.